In [1]:
import csv
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import datetime as dt 

In [198]:
# load movies names
movies = open('data/movies.txt')
movie_names =[]
for i in movies:
    movie_names.append(i)

In [121]:
# create matrix of nans then load ratings
matrix_m = np.empty(shape=(943, 1682)) * np.nan
train = open('data/ratings.csv', 'rb')
train = csv.reader(train)
for i in train:
    matrix_m[int(i[0])-1,int(i[1])-1] = i[2]

# probabilistic matrix factorization

In [610]:
count = float(np.sum(np.isnan(matrix_m)))/(matrix_m.shape[0]*matrix_m.shape[1])
count


0.9401056410398669

In [46]:
lamb = 1
varsq = .25

In [555]:
# coordinate ascent to build movie and user matrix factorizations
def matrix_fact(numb_factor, interations):
    user_list = []
    movie_list = []
    for i in range(interations): 
    
        user_m = np.random.multivariate_normal(np.zeros(numb_factor),np.eye(numb_factor),943)
        movie_m = np.random.multivariate_normal(np.zeros(numb_factor),np.eye(numb_factor),1682)
        first_part = lamb*varsq*np.eye(numb_factor)

        for t in range(0,100):

            for i in range(0,user_m.shape[0]):

                locas_nan = np.where(~np.isnan(matrix_m[i,:]))
                v_dot = np.dot(np.transpose(movie_m[locas_nan]),movie_m[locas_nan])
                m_v_dot = np.dot(matrix_m[i][locas_nan],movie_m[locas_nan])         
                user_m[i] = (np.dot(np.linalg.inv(first_part+v_dot),m_v_dot)).flatten()

            for k in range(0,movie_m.shape[0]):

                locas_nan = np.where(~np.isnan(matrix_m[:,k]))
                u_dot = np.dot(np.transpose(user_m[locas_nan]),user_m[locas_nan])
                m_u_dot = np.dot(matrix_m[:,k][locas_nan],user_m[locas_nan])
                movie_m[k] = (np.dot(np.linalg.inv(first_part+u_dot),m_u_dot)).flatten()
        
        user_list.append(user_m)
        movie_list.append(movie_m)
        
    return user_list, movie_list

In [556]:
user_list, movie_list = matrix_fact(10,10)

In [505]:
#calculate root meant squared error on test set
def RMSE(user_m, movie_m):    
    test = csv.reader(open('data/ratings_test.csv', 'rb'))
    RMSE = 0
    for i in test:        
        guess = np.dot(user_m[int(i[0])-1],movie_m[int(i[1])-1])
        RMSE += (float(i[2])-guess)**2
    RMSE_run = math.sqrt(RMSE/5000)
    return RMSE_run

In [557]:
RMSE_list = []
for i in range(10):
    RMSE_list.append(RMSE(user_list[i], movie_list[i]))
RMSE_list

[1.1137523331126953,
 1.1792417560966735,
 1.1146662114874695,
 1.0844383226978385,
 1.1458751009553745,
 1.1421008977227174,
 1.1036692650902695,
 1.0928419897808315,
 1.0946168075717735,
 1.0969677042118735]

In [558]:
#calculate movies that are most similar to given movie
def closest_movies(movie_numb, user_array, movie_array):
    dist=np.empty(shape=(movie_array.shape[0],))    
    for i in range(0,movie_array.shape[0]): 
        dist[i] = (np.linalg.norm(movie_array[movie_numb]-movie_array[i]))
    movie_order = np.argsort(dist)
    movies = []
    distances = []
    for i in range(1,11):
        movies.append(movie_names[movie_order[i]].split("(")[0].split(",")[0])
        distances.append(dist[movie_order[i]])
    return movies, distances

In [559]:
star_wars = closest_movies(49, user_list[3], movie_list[3])
my_fair_lady = closest_movies(484, user_list[3], movie_list[3])
goodfellas = closest_movies(181, user_list[3], movie_list[3])
air_force_one = closest_movies(299, user_list[3], movie_list[3])
fargo = closest_movies(99, user_list[3], movie_list[3])
jurassic_park = closest_movies(81, user_list[3], movie_list[3])
aladdin = closest_movies(94, user_list[3], movie_list[3])
lion_king = closest_movies(70, user_list[3], movie_list[3])
beauty_beast = closest_movies(587, user_list[3], movie_list[3])
toy_story = closest_movies(0, user_list[3], movie_list[3])
beavis = closest_movies(239, user_list[3], movie_list[3])
liar_liar = closest_movies(293, user_list[3], movie_list[3])
austin_power = closest_movies(248, user_list[3], movie_list[3])
braveheart = closest_movies(21, user_list[3], movie_list[3])
billy_madison = closest_movies(40, user_list[3], movie_list[3])
clueless = closest_movies(366, user_list[3], movie_list[3])
mission_impossible = closest_movies(404, user_list[3], movie_list[3])
batman = closest_movies(402, user_list[3], movie_list[3])
dumb_dumber = closest_movies(779, user_list[3], movie_list[3])

In [560]:
df = pd.DataFrame({'Beauty and the Beast' : dumb_dumber[0]})
df.index = df.index + 1
df

,Beauty and the Beast
1,Kingpin
2,Mother
3,Naked Gun 33 1/3: The Final Insult
4,Hackers
5,Tommy Boy
6,Ace Ventura: Pet Detective
7,Bulletproof
8,Happy Gilmore
9,Craft
10,Forget Paris


# what are the factors

In [588]:
most_rated_movies_for_factors = np.argsort(ratings_per_movie)[::-1][0:250]

In [601]:
movie_factor_table = pd.DataFrame()

movies_by_factor = []
column_order = np.argsort(movie_list[3], axis = 0)

for i in range(0,movie_list[0].shape[1]):
    
    top_5 = []
    counter1 = 0
    for j in range(500):
        if counter1 == 5:
            break
        elif column_order[::-1][j,i] in most_rated_movies_for_factors:
                top_5.append(column_order[::-1][j,i])
                counter1 += 1
    
    bottom_5 = []
    counter1 = 0
    for p in range(500):
        if counter1 == 5:
            break
        elif column_order[p,i] in most_rated_movies_for_factors:
                bottom_5.append(column_order[p,i])
                counter1 += 1           
    
    top_5_m = []
    bottom_5_m = []
    
    for h in top_5:
        top_5_m.append(movie_names[h].split("(")[0].split(",")[0])
    for q in bottom_5:
        bottom_5_m.append(movie_names[q].split("(")[0].split(",")[0])    

    temp = pd.DataFrame({'Factor '+str(i+1)+' Low': bottom_5_m,'Factor '+str(i+1)+' High': top_5_m})

    frames = [movie_factor_table,temp]
    
    movie_factor_table = pd.concat(frames,axis=1)
    
movie_factor_table.index = movie_factor_table.index + 1

In [602]:
movie_factor_table

,Factor 1 High,Factor 1 Low,Factor 2 High,Factor 2 Low,Factor 3 High,Factor 3 Low,Factor 4 High,Factor 4 Low,Factor 5 High,Factor 5 Low,Factor 6 High,Factor 6 Low,Factor 7 High,Factor 7 Low,Factor 8 High,Factor 8 Low,Factor 9 High,Factor 9 Low,Factor 10 High,Factor 10 Low
1,Crash,Boogie Nights,My Best Friend's Wedding,Boogie Nights,Mystery Science Theater 3000: The Movie,Crash,Everyone Says I Love You,Crash,Big Night,My Best Friend's Wedding,Reservoir Dogs,Sound of Music,Monty Python and the Holy Grail,Crash,Kingpin,In & Out,Remains of the Day,Everyone Says I Love You,Everyone Says I Love You,Die Hard: With a Vengeance
2,Die Hard: With a Vengeance,Everyone Says I Love You,Pretty Woman,This Is Spinal Tap,Fly Away Home,Boogie Nights,Evita,Starship Troopers,Monty Python's Life of Brian,Scream,Trainspotting,Gone with the Wind,Princess Bride,Jungle2Jungle,Beavis and Butt-head Do America,Contact,Manchurian Candidate,Spawn,Sound of Music,Top Gun
3,Star Trek IV: The Voyage Home,English Patient,First Wives Club,Clockwork Orange,Liar Liar,Leaving Las Vegas,Singin' in the Rain,Mars Attacks!,Dr. Strangelove or: How I Learned to Stop Worr...,Interview with the Vampire,Beavis and Butt-head Do America,It's a Wonderful Life,Star Wars,Evita,Mother,Everyone Says I Love You,Fargo,Mystery Science Theater 3000: The Movie,Mary Poppins,Rock
4,Die Hard 2,Clerks,Titanic,Mars Attacks!,Seven Years in Tibet,Mighty Aphrodite,Swingers,In & Out,Bridge on the River Kwai,Seven,Swingers,Sleepless in Seattle,Patton,Lost World: Jurassic Park,Ed Wood,Philadelphia,Some Like It Hot,George of the Jungle,Nightmare Before Christmas,Independence Day
5,Mystery Science Theater 3000: The Movie,Heat,Phenomenon,Monty Python and the Holy Grail,Saint,Wag the Dog,Jerry Maguire,Clockwork Orange,Annie Hall,Austin Powers: International Man of Mystery,Brazil,Ghost,Clerks,Spawn,Jungle2Jungle,Apt Pupil,Secrets & Lies,Beavis and Butt-head Do America,George of the Jungle,Time to Kill


# dimensionality reduction

In [623]:
from sklearn.manifold import TSNE
X = movie_list[7]
model = TSNE(n_components=2, perplexity = 5.0, learning_rate=200.0, n_iter = 2000, random_state=0)
np.set_printoptions(suppress=True)
movies_2d = model.fit_transform(X)

In [624]:
ratings_per_movie = np.sum(~np.isnan(matrix_m), axis=0)
most_rated_movies = np.argsort(ratings_per_movie)[::-1][0:20]
movie_labels = []
for i in most_rated_movies:
    movie_labels.append(movie_names[i].split("(")[0].split(",")[0])

In [625]:
df = pd.DataFrame({'X' : movies_2d[most_rated_movies][:,0], 'Y' : movies_2d[most_rated_movies][:,1], 'Movie' : movie_labels })
df.index = df.index + 1
df.to_csv('movie_dist.csv')
df

,Movie,X,Y
1,Star Wars,2.458152,8.254081
2,Contact,-2.927829,-6.689777
3,Return of the Jedi,2.503699,8.273936
4,Fargo,3.803853,-0.073355
5,Liar Liar,8.280136,4.030177
6,Scream,-0.686027,3.049107
7,English Patient,2.897108,-6.428655
8,Toy Story,0.823320,6.903513
9,Independence Day,-1.253729,4.534826
10,Raiders of the Lost Ark,2.381659,8.217517


In [ ]:
# fig, ax = plt.subplots()
# ax.set_xlim(-20,20)
# ax.set_ylim(-3,10)
# ax.scatter(x=movies_2d[most_rated_movies][:,0], y=movies_2d[most_rated_movies][:,1])
# for i, txt in exnumerate(movie_labels):
#     ax.annotate(txt, (movie_list[0][i,0],movie_list[0][i,1]))